# Livro para consulta:
- https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html
    

# 1. Importando bibliotecas <a name="import"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# 2. Carregando o dataframe SINASC <a name="read"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [ ]:
sinasc_raw = pd.read_csv('SINASC_RO_2019.csv')
sinasc_raw.head()

In [ ]:
sinasc_raw.info()

In [ ]:
print('Quantidade de linhas duplicadas:', 
      sinasc_raw.duplicated().sum())

# Tarefa 1

### 1. Idade media das mães e dos pais por município (coluna munResNome)


In [ ]:
sinasc_raw.groupby(by='munResNome')[['IDADEMAE', 
                                     'IDADEPAI']].agg({'IDADEMAE':[('Média','mean')],
                                                       'IDADEPAI':[('Média','mean')]})

### 2. Peso médio dos bebes por sexo que nasceram no dia do seu aniversário por faixas de escolaridade mae
Ex: Você, aluna(o), nasceu no dia 10/01, então você precisa filtrar o conjunto de dados nessa data e calcular o peso médio dos bebês de cada sexo por faixa de escolaridade da mãe.

In [ ]:
sinasc_raw.DTNASC = pd.to_datetime(sinasc_raw.DTNASC)
sinasc_raw.DTNASC

In [ ]:
sinasc_aniversario = sinasc_raw[(sinasc_raw.DTNASC.dt.day == 18) & 
                                (sinasc_raw.DTNASC.dt.month == 12)]

print('Em 2019, dos {} nascimentos registrados no Estado de Rondônia, {} ocorreram no dia 18 de dezembro.'.format(
    len(sinasc_raw), 
    len(sinasc_aniversario)
))

sinasc_aniversario = sinasc_aniversario.groupby(by=['DTNASC',
                                                    'SEXO',
                                                    'ESCMAE'])['PESO'].mean()

sinasc_aniversario = sinasc_aniversario.to_frame().reindex(labels=['1 a 3 anos', 
                                                                   '4 a 7 anos', 
                                                                   '8 a 11 anos', 
                                                                   '12 anos ou mais'], 
                                                           level='ESCMAE')

sinasc_aniversario

In [ ]:
data = sinasc_aniversario.index.get_level_values(level=0).unique()[0].date()

sns.catplot(x='ESCMAE', 
            y='PESO', 
            data=sinasc_aniversario.reset_index(level='DTNASC', 
                                                drop=True).reset_index(), 
            hue='SEXO', 
            kind='bar', 
            order=['1 a 3 anos', 
                   '4 a 7 anos', 
                   '8 a 11 anos', 
                   '12 anos ou mais'])
plt.xticks(rotation=45)
plt.xlabel(xlabel='Escolaridade da mãe')
plt.ylabel(ylabel='Média do peso em gramas')
plt.title(label=data.strftime('%d/%b/%Y'));

### 3. Qual o municipio que nasceu menos bebe em 2019?
    - qual a idade media, maxima, minima das maes nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?

In [ ]:
print('Municípios em que nasceram menos bebês:\n')
print(sinasc_raw.munResNome.value_counts().sort_values().head())

sinasc_raw['COUNT'] = 1
sinasc_3 = sinasc_raw.groupby(by='munResNome')[['COUNT', 
                                                'IDADEMAE',
                                                'IDADEPAI']].agg(
    {'COUNT':[('Quantidade de bebês nascidos','count')], 
     'IDADEMAE':[('idade média','mean'), 
                 ('idade máxima',max), 
                 ('idade mínima',min)], 
     'IDADEPAI':[('idade média','mean'), 
                 ('idade máxima',max), 
                 ('idade mínima',min)]})

sinasc_3.sort_values(('COUNT', 'Quantidade de bebês nascidos'))

### 4. Qual o municipio que nasceu mais bebe no mês de março?
    - qual a quantidade de filhos vivos media, maxima, minima nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?



### Analise as respostas encontradas, tire algum insight delas, conte pra gente algo encontrado nos dados. Algo que você julgue relevante e novo pra você.

Exemplo:
- Ah, descobri que a idade mediana das mulheres que deram a luz no ano de 2019 dos municipios x é maior que y.

In [6]:
sinasc_mar = sinasc_raw[sinasc_raw.DTNASC.dt.month == 3].copy()

print('O município que mais nasceu bebê no mês de março foi {} com {} nascimentos.'.format(
    sinasc_mar['munResNome'].value_counts().index[0], 
    sinasc_mar['munResNome'].value_counts().values[0]))

sinasc_mar['MES'] = sinasc_mar['DTNASC'].dt.month_name()
sinasc_mar['COUNT'] = 1
sinasc_4 = sinasc_mar.groupby(by=['MES',
                                  'munResNome']
                             )[['COUNT', 
                                'QTDFILVIVO',
                                'IDADEMAE',
                                'IDADEPAI']
                              ].agg({'COUNT':[('Qtd de bebês nascidos','count')], 
                                     'QTDFILVIVO':[('Média','mean'), 
                                                   ('Máx',max), 
                                                   ('Mín',min)], 
                                     'IDADEMAE':[('Média','mean'), 
                                                 ('Máx',max), 
                                                 ('Mín',min), 
                                                 ('Amplitude',lambda x: x.max()-x.min())], 
                                     'IDADEPAI':[('Média','mean'), 
                                                 ('Máx',max), 
                                                 ('Mín',min), 
                                                 ('Amplitude',lambda x: x.max()-x.min())]
                                    })

sinasc_4['Diferença da média IDADEPAI-IDADEMAE'] = sinasc_4[('IDADEPAI','Média')]-sinasc_4[('IDADEMAE','Média')]
sinasc_4.sort_values(('COUNT', 'Qtd de bebês nascidos'), 
                     ascending=False
                    )

ORIGEM          int64
CODESTAB      float64
CODMUNNASC      int64
LOCNASC         int64
IDADEMAE        int64
               ...   
munResUf       object
munResLat     float64
munResLon     float64
munResAlt     float64
munResArea    float64
Length: 69, dtype: object

In [7]:
print('Mediana de todos os municípios no mês de março:')
round(sinasc_4.groupby(by='MES').median())

,ESCMAE,GESTACAO,GRAVIDEZ,PARTO,DTNASC,SEXO,RACACOR,DTCADASTRO,CODANOMAL,VERSAOSIST,DTRECEBIM,munResStatus,munResTipo,munResNome,munResUf
0,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-19,Masculino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
1,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-21,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
2,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-25,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
3,12 anos ou mais,37 a 41 semanas,Única,Cesáreo,2019-03-20,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia
4,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-03-23,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,12 anos ou mais,32 a 36 semanas,Única,Cesáreo,2019-12-13,Masculino,Branca,2020-09-28,NaN,3.2.01,2020-10-13,ATIVO,MUNIC,Vilhena,Rondônia
27024,8 a 11 anos,37 a 41 semanas,Única,Vaginal,2019-10-04,Masculino,Branca,2019-10-16,NaN,3.2.01,2019-10-21,ATIVO,MUNIC,Chupinguaia,Rondônia
27025,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-08-02,Masculino,Parda,2019-08-07,NaN,3.2.00,2019-08-07,ATIVO,MUNIC,Vilhena,Rondônia
27026,8 a 11 anos,32 a 36 semanas,Única,Vaginal,2019-12-23,Masculino,Parda,2020-01-07,NaN,3.2.00,2020-01-07,ATIVO,MUNIC,Vilhena,Rondônia


In [4]:
print('Média de todos os municípios no mês de março:')
round(sinasc_4.groupby(by='MES').mean())

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')